In [12]:
import oracledb
import pandas as pd
import sys
import os
sys.path.append(os.path.abspath("../src"))
from datetime import datetime
from dateutil import parser


In [13]:
df = pd.read_csv("../data/vader_sentiment_reviews.csv")

In [14]:
df

,review,rating,date,bank,source,processed_review,sentiment_label,sentiment_score
0,20 years,5,2025-06-08,CBE,Google Play,20 year,neutral,0.0000
1,A great app. It's like carrying a bank in your...,4,2025-06-07,CBE,Google Play,great app like carrying bank pocket,positive,0.7650
2,More than garrantty bank EBC.,4,2025-06-07,CBE,Google Play,garrantty bank ebc,neutral,0.0000
3,really am happy to this app it is Siple to use...,5,2025-06-07,CBE,Google Play,really happy app siple use everything,positive,0.6115
4,I liked this app. But the User interface is ve...,2,2025-06-07,CBE,Google Play,liked app user interface basic attractive,positive,0.6908
...,...,...,...,...,...,...,...,...
1195,Wow,5,2025-01-17,Dashen,Google Play,wow,positive,0.5859
1196,Dashen yichalal. Ewnetem one step a head,5,2025-01-17,Dashen,Google Play,dashen yichalal ewnetem one step head,neutral,0.0000
1197,It has a Good performance but need more upgrad...,4,2025-01-17,Dashen,Google Play,good performance need upgrade performance like...,positive,0.9371
1198,It is a very wonderful work that has saved its...,5,2025-01-17,Dashen,Google Play,wonderful work saved time always one step ahead,positive,0.7579


#### Extract the unique banks from the dataframe 

In [15]:
bank_names = df["bank"].unique()

In [16]:
bank_names

array(['CBE', 'BoA', 'Dashen'], dtype=object)

##### Connect to Oracle DB

In [17]:
# Connect to Oracle DB
connection = oracledb.connect(
    user="bank_reviews",
    password="strongpassword",
    dsn="localhost/XEPDB1"
)
cursor = connection.cursor()
# Insert these banks into Banks table once, and map their IDs
bank_id_map = {}

for name in bank_names:
    # 1. Check if the bank already exists
    cursor.execute("SELECT bank_id FROM Banks WHERE bank_name = :1", (name,))
    existing_bank = cursor.fetchone()
    if existing_bank:
        # If bank exists, use its existing bank_id
        bank_id = existing_bank[0]
        print(f"Bank '{name}' already exists with ID: {bank_id}.")
        bank_id_map[name] = bank_id
    else:
        # Insert bank name into Banks table
        cursor.execute("INSERT INTO Banks (bank_name) VALUES (:1)", (name,))
            
        # Retrieve the generated bank_id for this bank
        cursor.execute("SELECT bank_id FROM Banks WHERE bank_name = :1", (name,))
        bank_id = cursor.fetchone()[0]
            
        # Save the mapping for later use in Reviews insertion
        bank_id_map[name] = bank_id

# Commit all changes
connection.commit()

# Close cursor and connection
cursor.close()
connection.close()


Bank 'CBE' already exists with ID: 1.
Bank 'BoA' already exists with ID: 2.
Bank 'Dashen' already exists with ID: 3.


In [18]:
print(bank_id_map)

{'CBE': 1, 'BoA': 2, 'Dashen': 3}


In [19]:
# Connect to Oracle DB
connection = oracledb.connect(
    user="bank_reviews",
    password="strongpassword",
    dsn="localhost/XEPDB1"
)
cursor = connection.cursor()

for _, row in df.iterrows():
    try:
        # Flexible date parsing that handles both formats:
        review_date = parser.parse(row["date"]).date() if row["date"] else None
        
        cursor.execute("""
            INSERT INTO Reviews (
                bank_id, review_text, processed_review_data,
                rating, review_date, sentiment_label, sentiment_score
            ) VALUES (
                :1, :2, :3, :4, :5, :6, :7
            )
        """, (
            int(bank_id_map[row["bank"]]),
            row["review"],
            row["processed_review"] if pd.notna(row["processed_review"]) else None,
            float(row["rating"]),
            review_date,  # Pass as Python date object
            row["sentiment_label"],
            float(row["sentiment_score"])
        ))
    except Exception as e:
        print(f"Failed row: {dict(row)}")
        print(f"Error: {str(e)}")
        raise

connection.commit()

# Close cursor and connection
cursor.close()
connection.close()